In [5]:
import os
from random import random, randint
from mlflow import log_metric, log_param, log_artifacts

mlflow.set_tracking_uri("http://aiserver.tzp.haw-landshut.de:5000")
mlflow.set_experiment("my-experiment1")

# Log a parameter (key-value pair)
log_param("param1", randint(0, 100))

# Log a metric; metrics can be updated throughout the run
log_metric("foo", random())
log_metric("foo", random() + 1)
log_metric("foo", random() + 2)

# Log an artifact (output file)
if not os.path.exists("outputs"):
    os.makedirs("outputs")
with open("outputs/test.txt", "w") as f:
    f.write("hello world!")
log_artifacts("outputs")

2023/03/15 15:44:07 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment1' does not exist. Creating a new experiment.


MlflowException: INVALID_PARAMETER_VALUE: Changing param values is not allowed. Param with key='param1' was already logged with value='91' for run ID='35d7de9f18924195890050a7e23a3397'. Attempted logging new value '59'.

The cause of this error is typically due to repeated calls
to an individual run_id event logging.

Incorrect Example:
---------------------------------------
with mlflow.start_run():
    mlflow.log_param("depth", 3)
    mlflow.log_param("depth", 5)
---------------------------------------

Which will throw an MlflowException for overwriting a
logged parameter.

Correct Example:
---------------------------------------
with mlflow.start_run():
    with mlflow.start_run(nested=True):
        mlflow.log_param("depth", 3)
    with mlflow.start_run(nested=True):
        mlflow.log_param("depth", 5)
---------------------------------------

Which will create a new nested run for each individual
model and prevent parameter key collisions within the
tracking store.'